# Olympic Games support code

### What's inside

This notebook contains all the Python code used to manipulate the original datasets in order to produce additional datasets used for producing the various visualizations

### Imports

The following block contains all the necessary imports.

In [32]:
import numpy as np
import pandas as pd
import csv
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.mlab import PCA as mlabPCA
from sklearn import preprocessing

### Data reading an importing

In the following block we read our datasets and we transform them into data structures which can be scanned and manipulated in Python.

In [33]:
att=['id','name','sex','age','height',
     'weight','team','noc','games','year','season',
     'city','sport','event','medal']

data = pd.io.parsers.read_csv('../main_dataset/athlete_events.csv',
     delimiter=',',
     header=None,
     usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14],
     
     skiprows=[0]
    )
data.columns = att
##print(data.head())

## The following line substitutes null values with the string NA
data = data.fillna('NA')

d = data.values

noc = pd.read_csv("../main_dataset/noc_regions.csv", header='infer')
noc_val = noc.values
##print(noc.head())

#print(type(noc))
#print(type(noc.values))
#print(data['name'])

#sns.heatmap(data.isnull(), cbar=False)

### Creation of differenciated datasets

In the following block we divide the original dataset in many other datasets according to some characteristics.

Description of the files' content:
- s: this file contains all tuples from Summer Olympic Games (including incomplete tuples)
- w: this file contains all tuples from Winter Olympic Games (including incomplete tuples)
- sc: it only contains complete tuples from Summer Olympic Games
- wc: it only contains complete tuples from Winter Olympic Games
- sm: it only contains those tuples from Summer Games (even incomplete ones) where a medal was won
- wm: it only contains those tuples from Winter Games (even incomplete ones) where a medal was won

In [34]:


## create files
sm = open("../proj_datasets/summer_medals.csv", "w+")

writer_sm = csv.writer(sm, lineterminator='\n')

writer_sm.writerow(['id','name','sex','age','height','weight','noc','year','season','city','sport','event','medal'])

for i in range(0, len(d)-1):     
    ## unify nocs
    if d[i][7]=='FRG' or d[i][7]=='GDR' or d[i][7]=='SAA':
        d[i][7] = 'GER'
    elif d[i][7]=='EUN' or d[i][7]=='URS':
        d[i][7] = 'RUS'
    elif d[i][7]=='ANZ':
        d[i][7] = 'AUS'
    elif d[i][7]=='HKG':
        d[i][7] = 'CHN'
    elif d[i][7]=='BOH' or d[i][7]=='TCH':
        d[i][7] = 'CZE'
    elif d[i][7]=='MAS' or d[i][7]=='NBO':
        d[i][7] = 'MAL'
    elif d[i][7]=='SCG' or d[i][7]=='YUG':
        d[i][7] = 'SRB'
    elif d[i][7]=='UAR':
        d[i][7] = 'SYR'
    elif d[i][7]=='WIF':
        d[i][7] = 'TTO'
    elif d[i][7]=='VIE':
        d[i][7] = 'VNM'
    elif d[i][7]=='YAR' or d[i][7]=='YMD':
        d[i][7] = 'YEM'
    elif d[i][7]=='RHO':
        d[i][7] = 'ZIM'
        
    
    if d[i][10]=='Summer':
        if d[i][14]!='NA':
            ## HERE I HAVE ALL SUMMER MEDALS, ALSO WITH INCOMPLETE DATA FOR THE ATHLETES
            s_medals +=1
            writer_sm.writerow([d[i][0], d[i][1], d[i][2], d[i][3], d[i][4], d[i][5], d[i][7], d[i][9], d[i][10],
                         d[i][11], d[i][12], d[i][13], d[i][14]])
              
        
sm.close()

### Creation of a dictionary for summer medals

Here we are focusing on just the file of summer medals in order to create a dictionary of it.
In the meanwhile, we also create a dictionary (dis_dic) which will be used to create a file for the RadarChart viz later on.

In [35]:
## Use summer_medals.csv to generate the JSON for the linechart
## Alternatives: number of medals

sum_med_dic = {}

## editions of the games
years = list(range(1896, 2017, 4))
years.append(1906)
years.remove(1916)
years.remove(1940)
years.remove(1944)

years.sort()

att_sum_med=['id','name','sex','age','height',
     'weight','noc','year','season',
     'city','sport','event', 'medal']

data_sum_med = pd.io.parsers.read_csv('../proj_datasets/summer_medals.csv',
     delimiter=',',
     header=None,
     usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12],    
     skiprows=[0]
    )
data_sum_med.columns = att_sum_med
## The following line substitutes null values with the string NA
data_sum_med = data_sum_med.fillna('NA')

d_sum_med = data_sum_med.values

teams = list()


dis_dic = {}
dis_set = set()
disciplines = ['Swimming', 'Gymnastics', 'Hockey', 'Rowing', 'Football', 'Sailing', 'Cycling', 'Fencing', 'Athletics', 'Canoeing', 'Wrestling', 'Shooting']

# initialize dis_dic for all nations (nocs) and disciplines
for i in range(0, len(noc_val)):
    cntr = noc_val[i][0]
    if cntr not in dis_dic:
        dis_dic[cntr] = {}
        for j in sorted(disciplines):
            dis_dic[cntr][j] = 0
    else:
        for j in sorted(disciplines):
            dis_dic[cntr][j] = 0
#print(dis_dic)
        
print(d_sum_med[0][9])

years_cities_dic = {}
for i in range(0, len(d_sum_med)-1):
    yr = d_sum_med[i][7]
    ct = d_sum_med[i][9]
    
    if yr not in years_cities_dic:
        years_cities_dic[yr] = ct

# print(data_sum_med.head)
for i in range(0, len(d_sum_med)-1):    
    noc_temp = d_sum_med[i][6]
    year_temp = d_sum_med[i][7]
    city_temp = d_sum_med[i][9]
    
    disc = d_sum_med[i][10]
    
    dis_set.add(d_sum_med[i][10])
    
    if disc in disciplines:
        dis_dic[noc_temp][disc] += 1
    
    # combination of noc,year,event,medal
    s = d_sum_med[i][6]+str(d_sum_med[i][7])+d_sum_med[i][11]+d_sum_med[i][12]
    #print(s)
    
    ##if noc_temp=='USA' and year_temp==1900:
    ##        print(d_sum_med[i][0], '-',d_sum_med[i][1], '-', d_sum_med[i][7], '-', d_sum_med[i][11], '-', d_sum_med[i][12])
   
    #if s not in teams and d_sum_med[i][10]!='Art Competitions':
    ## begin of the 'teams' and art conditional block
        #teams.append(s)
        
    noc_temp = d_sum_med[i][6]
    year_temp = d_sum_med[i][7]
        
    #if noc_temp=='AUS' and year_temp==1896:
    #    print(d_sum_med[i][0], '-',d_sum_med[i][1], '-', d_sum_med[i][7], '-', d_sum_med[i][11], '-', d_sum_med[i][12])
        
    # if new NOC...
    if noc_temp not in sum_med_dic:
        #insert it into the dictionary
        sum_med_dic[noc_temp] = {}
        #add a key for every olympic games edition
        for k in years:
            sum_med_dic[noc_temp][k] = [0, '']
        #add a medal for the corresponding year
        sum_med_dic[noc_temp][year_temp][0] += 1
        sum_med_dic[noc_temp][year_temp][1] = years_cities_dic[year_temp]
            
    # if the NOC exists in the dictionary...
    else:
        sum_med_dic[noc_temp][year_temp][0] += 1
        sum_med_dic[noc_temp][year_temp][1] = years_cities_dic[year_temp]
    ## end of the 'teams' and art conditional block
            
                

n_c_dic = {}
number = 0

for i in sum_med_dic:
    
    if i not in n_c_dic:
        n_c_dic[i] = number
        number += 1
    
    
    for j in sum_med_dic[i]:
        sum_med_dic[i][j][1] = years_cities_dic[j]
            
#print(sum_med_dic['ITA'])

## Now the dictionary has as primary key the NOC of the country, and as second key the year
## For each noc and for each year I have the total amount of medals won
print()
print(sum_med_dic['ITA'])

#print('---')
#print('keys:', len(sport_dic))
#for i in sport_dic:
#    if sport_dic[i] >= 1150:
#        print(i, ':', sport_dic[i])
        
#print(dis_dic['ITA'])

#print('DISCIPLINES')
#print(len(dis_set))
#print(dis_set)

Paris

{1896: [0, 'Athina'], 1900: [5, 'Paris'], 1904: [0, 'St. Louis'], 1906: [56, 'Athina'], 1908: [8, 'London'], 1912: [25, 'Stockholm'], 1920: [84, 'Antwerpen'], 1924: [51, 'Paris'], 1928: [72, 'Amsterdam'], 1932: [77, 'Los Angeles'], 1936: [76, 'Berlin'], 1948: [67, 'London'], 1952: [54, 'Helsinki'], 1956: [47, 'Melbourne'], 1960: [88, 'Roma'], 1964: [51, 'Tokyo'], 1968: [33, 'Mexico City'], 1972: [25, 'Munich'], 1976: [31, 'Montreal'], 1980: [37, 'Moskva'], 1984: [63, 'Los Angeles'], 1988: [29, 'Seoul'], 1992: [45, 'Barcelona'], 1996: [71, 'Atlanta'], 2000: [65, 'Sydney'], 2004: [104, 'Athina'], 2008: [42, 'Beijing'], 2012: [68, 'London'], 2016: [72, 'Rio de Janeiro']}


## Radar Chart data (file)

In the following block we output the content of the dictionary about amount of medals in each discipline for each nation in a file which will be used for the radar chart

In [36]:
## Now I should produce a JSON of the dictionary that can be used by the js code

dis_file = open('../proj_datasets/disciplines.txt', 'w+')
content = ''
n_country = 0

dis_file.write('var disciplines = [\n')

for i in dis_dic:
    dis_file.write('{\n\t')
    dis_file.write('id: "' + i + '",\n\t')
    dis_file.write('name: "' + i + '",\n\t')
    dis_file.write('values: [\n\t\t')
    for j in dis_dic[i]:
        dis_file.write('{axis:"' + j + '",value:' + str(dis_dic[i][j]) + '},\n\t\t')
    dis_file.write(']},')
dis_file.write(']')
    

    
#print(content)
dis_file.write(content)
dis_file.close()

## LineChart data (file)

The following block produces the content of a variable which will be used for the LineChart.
It contains, for each country, the year and the relative amount of medals won by that specific country.

In [37]:
## Now I should produce a JSON of the dictionary that can be used by the js code

json = open('../proj_datasets/json.txt', 'w+')
content = ''
n_country = 0


## NOTE: include the city in sum_med_dic (for each year)
print(sum_med_dic['ITA'])

for i in sorted(sum_med_dic):
    nation = ''
    for k in range(0, len(noc_val)):
        if(noc_val[k][0]==i):
            nation = noc_val[k][1]
    
    content = content + '{\n\tid: '+'"'+ i +'"'+',\n\t'+'name: '+'"'+ nation +'"'+',\n\t'+'n_country: '+str(n_country)+',\n\t'+'values: [\n'
    n_country += 1
    for j in sum_med_dic[i]:
        if j!=2016:
            #content = content + '\t\t{date:'+str(j)+',rank:'+str(sum_med_dic[i][j])+'},\n'
            content = content + '\t\t{date:'+str(j)+',city:"'+sum_med_dic[i][j][1]+'",rank:'+str(sum_med_dic[i][j][0])+'},\n'
        else:
            #content = content + '\t\t{date:'+str(j)+',rank:'+str(sum_med_dic[i][j])+'}\n'
            content = content + '\t\t{date:'+str(j)+',city:"'+sum_med_dic[i][j][1]+'",rank:'+str(sum_med_dic[i][j][0])+'}\n'
            
            
    if i!='ZIM':
        content = content + '\t]\n},\n'
    else:
        content = content + '\t]\n}'
    
    
#insert 0 medals countries

##for i in range(0, len(noc_val)):
##    cntr = noc_val[i][0]
##    if cntr not in sum_med_dic:
##        print('{\n\tid: "' + cntr + '",\n\tname: "' + noc_val[i][1] + '",n_country: '+str(n_country)+',\n\tvalues: [')
##        n_country += 1
##        for j in sum_med_dic['ITA']:
##            if j!=2016:
##                print('\t\t{date:'+str(j)+',city:"'+sum_med_dic['ITA'][j][1]+'",rank:0},')
##            else:
##                print('\t\t{date:'+str(j)+',city:"'+sum_med_dic['ITA'][j][1]+'",rank:0}')
##        
##        print('\t]\n},')
##                

#print(content)
json.write(content)
json.close()

{1896: [0, 'Athina'], 1900: [5, 'Paris'], 1904: [0, 'St. Louis'], 1906: [56, 'Athina'], 1908: [8, 'London'], 1912: [25, 'Stockholm'], 1920: [84, 'Antwerpen'], 1924: [51, 'Paris'], 1928: [72, 'Amsterdam'], 1932: [77, 'Los Angeles'], 1936: [76, 'Berlin'], 1948: [67, 'London'], 1952: [54, 'Helsinki'], 1956: [47, 'Melbourne'], 1960: [88, 'Roma'], 1964: [51, 'Tokyo'], 1968: [33, 'Mexico City'], 1972: [25, 'Munich'], 1976: [31, 'Montreal'], 1980: [37, 'Moskva'], 1984: [63, 'Los Angeles'], 1988: [29, 'Seoul'], 1992: [45, 'Barcelona'], 1996: [71, 'Atlanta'], 2000: [65, 'Sydney'], 2004: [104, 'Athina'], 2008: [42, 'Beijing'], 2012: [68, 'London'], 2016: [72, 'Rio de Janeiro']}


## Choropleth data (file)

The following block produces a csv file which is used for producing the Choropleth map.
The file contains names, nocs and total amount of won medals for each country.

In [38]:
with open('../proj_datasets/med_amount.csv', 'w+') as meds_for_choro:
    writer = csv.writer(meds_for_choro, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['name', 'code', 'medals'])
        
    counter = 0
    for i in sum_med_dic:
        for j in sum_med_dic[i]:
            counter += sum_med_dic[i][j][0]
        writer.writerow([i, i, counter])
        counter = 0

## Building the dataset for Dimensionality Reduction

The idea is to create a table with one line per winning country (1 medal is sufficient) and 18 features:
- age
- height
- weight
- gold medals
- silver medals
- athletics medals
- canoeing medals
- cycling medals
- football medals
- gymnastic medals
- hockey medals
- rowing medals
- sailing medals
- shooting medals
- swimming medals
- wrestling medals

This dataset will be given as input to a dimensionality reduction function.

In [39]:
dim_red_dic = {}

att_sum_med=['id','name','sex','age','height',
     'weight','noc','year','season',
     'city','sport','event', 'medal']

data_sum_med = pd.io.parsers.read_csv('../proj_datasets/summer_medals.csv',
     delimiter=',',
     header=None,
     usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12],    
     skiprows=[0]
    )
data_sum_med.columns = att_sum_med
## The following line substitutes null values with the string NA
data_sum_med = data_sum_med.fillna('NA')

d_sum_med = data_sum_med.values

##for i in range(0, len(d_sum_med)):
##    ## analyze medal by medal (take age, height, weight, noc)
##    print(d_sum_med[i])

for i in range(0, len(d_sum_med)):    
    t_age = d_sum_med[i][3]
    t_height = d_sum_med[i][4]
    t_weight = d_sum_med[i][5]
    t_noc = d_sum_med[i][6]
    t_medal = d_sum_med[i][12]
    t_disc = d_sum_med[i][10]
    
    if t_disc in disciplines:
        t_disc = t_disc.lower()
        
        if t_noc in dim_red_dic:
            ## update values
            if t_age != 'NA':
                dim_red_dic[t_noc]['age'] += t_age
                dim_red_dic[t_noc]['age_counter'] += 1
            if t_height != 'NA':
                dim_red_dic[t_noc]['height'] += t_height
                dim_red_dic[t_noc]['height_counter'] += 1
            if t_weight != 'NA':
                dim_red_dic[t_noc]['weight'] += t_weight
                dim_red_dic[t_noc]['weight_counter'] += 1
            dim_red_dic[t_noc]['medals'] += 1
            if t_medal == 'Gold':
                dim_red_dic[t_noc]['global_gold'] += 1
                dim_red_dic[t_noc][t_disc+'_gold'] += 1
            elif t_medal == 'Silver':
                dim_red_dic[t_noc]['global_silver'] += 1
                dim_red_dic[t_noc][t_disc+'_silver'] += 1
            elif t_medal == 'Bronze':
                dim_red_dic[t_noc]['global_bronze'] += 1
                dim_red_dic[t_noc][t_disc+'_bronze'] += 1
                
        else:
            #new noc -> initialize
            dim_red_dic[t_noc] = {}
            
            dim_red_dic[t_noc]['age'] = 0
            dim_red_dic[t_noc]['age_counter'] = 0
            if t_age != 'NA':
                dim_red_dic[t_noc]['age'] += t_age
                dim_red_dic[t_noc]['age_counter'] += 1
                
            dim_red_dic[t_noc]['height'] = 0
            dim_red_dic[t_noc]['height_counter'] = 0
            if t_height != 'NA':
                dim_red_dic[t_noc]['height'] += t_height
                dim_red_dic[t_noc]['height_counter'] += 1
                
            dim_red_dic[t_noc]['weight'] = 0
            dim_red_dic[t_noc]['weight_counter'] = 0   
            if t_weight != 'NA':
                dim_red_dic[t_noc]['weight'] += t_weight
                dim_red_dic[t_noc]['weight_counter'] += 1
            
            dim_red_dic[t_noc]['medals'] = 1
            dim_red_dic[t_noc]['global_gold'] = 0
            dim_red_dic[t_noc]['global_silver'] = 0
            dim_red_dic[t_noc]['global_bronze'] = 0
            
            for d in disciplines:
                dim_red_dic[t_noc][d.lower()+'_gold'] = 0
                dim_red_dic[t_noc][d.lower()+'_silver'] = 0
                dim_red_dic[t_noc][d.lower()+'_bronze'] = 0
            
            if t_medal == 'Gold':
                dim_red_dic[t_noc]['global_gold'] += 1
                dim_red_dic[t_noc][t_disc+'_gold'] += 1
            elif t_medal == 'Silver':
                dim_red_dic[t_noc]['global_silver'] += 1
                dim_red_dic[t_noc][t_disc+'_silver'] += 1
            elif t_medal == 'Bronze':
                dim_red_dic[t_noc]['global_bronze'] += 1
                dim_red_dic[t_noc][t_disc+'_bronze'] += 1
            
          
            
dr = open("../proj_datasets/dimensionality_reduction_input.csv", "w+")
writer_dr = csv.writer(dr, lineterminator='\n')

dis_features = ''
dis_parameters = ''
for k in disciplines:
    kl = k.lower()
    dis_features += "'"+kl+"_gold',"+"'"+kl+"_silver',"+"'"+kl+"_bronze',"
    dis_parameters += "dim_red_dic[i]['"+kl+"_gold'],dim_red_dic[i]['"+kl+"_silver'],dim_red_dic[i]['"+kl+"_bronze'],"

writer_dr.writerow(['noc','avg_age', 'avg_height', 'avg_weight', 'medals',
                    'global_gold','global_silver','global_bronze',
                    'swimming_gold','swimming_silver','swimming_bronze',
                    'gymnastics_gold','gymnastics_silver','gymnastics_bronze',
                    'hockey_gold','hockey_silver','hockey_bronze',
                    'rowing_gold','rowing_silver','rowing_bronze',
                    'football_gold','football_silver','football_bronze',
                    'sailing_gold','sailing_silver','sailing_bronze',
                    'cycling_gold','cycling_silver','cycling_bronze',
                    'fencing_gold','fencing_silver','fencing_bronze',
                    'athletics_gold','athletics_silver','athletics_bronze',
                    'canoeing_gold','canoeing_silver','canoeing_bronze',
                    'wrestling_gold','wrestling_silver','wrestling_bronze',
                    'shooting_gold','shooting_silver','shooting_bronze'])
    
    

#print(dis_features)
#print(dis_parameters)


#print(dim_red_dic['USA'])


for i in dim_red_dic:
    #print(dim_red_dic[i])
    t_noc = i
    
    if dim_red_dic[i]['height_counter'] == 0:
        t_height = 175.0
    else:
        t_height = round(dim_red_dic[i]['height']/(dim_red_dic[i]['height_counter']), 2)
    
    if dim_red_dic[i]['weight_counter'] == 0:
        t_weight = 75.0
    else:
        t_weight = round(dim_red_dic[i]['weight']/(dim_red_dic[i]['weight_counter']), 2)
    
    t_age = int(dim_red_dic[i]['age']/(dim_red_dic[i]['age_counter']))
    t_medals = dim_red_dic[i]['medals']
    t_gold = dim_red_dic[i]['global_gold']
    t_silver = dim_red_dic[i]['global_silver']
    t_bronze = dim_red_dic[i]['global_bronze']
    
    writer_dr.writerow([t_noc, t_age, t_height, t_weight, t_medals, t_gold, t_silver, t_bronze,
                       dim_red_dic[i]['swimming_gold'],dim_red_dic[i]['swimming_silver'],dim_red_dic[i]['swimming_bronze'],
                        dim_red_dic[i]['gymnastics_gold'],dim_red_dic[i]['gymnastics_silver'],dim_red_dic[i]['gymnastics_bronze'],
                        dim_red_dic[i]['hockey_gold'],dim_red_dic[i]['hockey_silver'],dim_red_dic[i]['hockey_bronze'],
                        dim_red_dic[i]['rowing_gold'],dim_red_dic[i]['rowing_silver'],dim_red_dic[i]['rowing_bronze'],
                        dim_red_dic[i]['football_gold'],dim_red_dic[i]['football_silver'],dim_red_dic[i]['football_bronze'],
                        dim_red_dic[i]['sailing_gold'],dim_red_dic[i]['sailing_silver'],dim_red_dic[i]['sailing_bronze'],
                        dim_red_dic[i]['cycling_gold'],dim_red_dic[i]['cycling_silver'],dim_red_dic[i]['cycling_bronze'],
                        dim_red_dic[i]['fencing_gold'],dim_red_dic[i]['fencing_silver'],dim_red_dic[i]['fencing_bronze'],
                        dim_red_dic[i]['athletics_gold'],dim_red_dic[i]['athletics_silver'],dim_red_dic[i]['athletics_bronze'],
                        dim_red_dic[i]['canoeing_gold'],dim_red_dic[i]['canoeing_silver'],dim_red_dic[i]['canoeing_bronze'],
                        dim_red_dic[i]['wrestling_gold'],dim_red_dic[i]['wrestling_silver'],dim_red_dic[i]['wrestling_bronze'],
                        dim_red_dic[i]['shooting_gold'],dim_red_dic[i]['shooting_silver'],dim_red_dic[i]['shooting_bronze']])


dr.close()